In [1]:
import bs4, requests, re
from urllib.request import Request, urlopen
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions
from bs4 import BeautifulSoup
import numpy as np
from selenium.webdriver.support.wait import WebDriverWait
import time
import random

In [6]:
airline_info_pages = pd.read_excel('airline_info_pages.xlsx', names = ['stt','links'])

In [11]:
airline_info_pages['links']

0       https://www.airfleets.net/flottecie/Armenia Ai...
1       https://www.airfleets.net/flottecie/Hong Kong ...
2       https://www.airfleets.net/flottecie/Felix Airw...
3       https://www.airfleets.net/flottecie/Polish Gov...
4       https://www.airfleets.net/flottecie/Transcarga...
                              ...                        
1286    https://www.airfleets.net/flottecie/Estelar La...
1287    https://www.airfleets.net/flottecie/Smartwings...
1288    https://www.airfleets.net/flottecie/Eastar Jet...
1289    https://www.airfleets.net/flottecie/Air Georgi...
1290       https://www.airfleets.net/flottecie/Novair.htm
Name: links, Length: 1291, dtype: object

In [32]:
result = pd.DataFrame(columns = ['Aircraft','Active','Parked','Stored / Scrapped','History','On order','country','code'])
ac_quantity = ['Active','Parked','Stored / Scrapped','History','On order']

In [33]:
for page in airline_info_pages['links']:
    driver = webdriver.Chrome("chromedriver.exe")
    driver.get(page)
    WebDriverWait(page, random.randint(5,10))
    driver.implicitly_wait(random.randint(5,10))
    clean = re.sub('<br>','|', str(driver.page_source))
    ac_soup = BeautifulSoup(clean,'lxml')
    time.sleep(random.randint(4,5))
    ac_group = []
    ac = ac_soup.find_all('a',attrs = {'class':'lienn'})
    for i in ac:
        ac_group.append(i.text)
    df = pd.read_html(clean, attrs = {'class':'tab800'})[0]
    df.columns=df.iloc[1]
    df.drop(df.index[:2], inplace=True)
    df.drop(df.index[-1:], inplace=True)
    df = df.applymap(lambda x: x.split("|"))
    df[ac_quantity]=df[ac_quantity].applymap(lambda x:x[:-1]) 
    df.drop(columns = ['Total','Age'], inplace = True)
    df = df.explode(['Aircraft','Active','Parked','Stored / Scrapped','History','On order'])
    df = df[~df['Aircraft'].isin(ac_group)]
    
    # get the country where the airlines base
    co = ac_soup.find('a', href = re.compile("/recherche/list-country")).text
    df['country'] = co
    
    # get IATA codes of the airlines
    tableau = pd.read_html(clean, attrs = {'class':'tableau'})[0]
    code = tableau.iloc[tableau[tableau[0]=='Codes'].index.values][1].values
    df['code'] = code[0]
    
    result = result.append(df)

C:\Users\NAMTRA~1\AppData\Local\Temp/ipykernel_48692/3476561852.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe")


ValueError: No tables found

In [34]:
result

,Aircraft,Active,Parked,Stored / Scrapped,History,On order,country,code
2,Airbus A310-300,,,,2,,Armenia,6A AMW
3,Boeing 737-300,1,,,,,Armenia,6A AMW
3,Boeing 737-500,1,,,,,Armenia,6A AMW
4,Bae 146,,1,,,,Armenia,6A AMW
2,Airbus A320 Neo,4,6,,,,Hong-Kong,UO HKE
...,...,...,...,...,...,...,...,...
7,McDonnell MD-80/90,,,,1,,Ukraine,PS AUI
7,McDonnell MD-82,,,,1,,Ukraine,PS AUI
3,Embraer 135,1,,,,,France,PEA
3,Embraer 145,1,,,,,France,PEA


In [35]:
result.to_excel('airfleets.xlsx')